# Word Embeddings : le modèle Word2Vec

## Imports

In [25]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [26]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [27]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [28]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [29]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [30]:
len(bigram_phrases.vocab.keys())

815775

Prenons une clé au hasard :

In [31]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

taxes_communale


Le dictionnaire indique le score de cette coocurrence :

In [32]:
bigram_phrases.vocab[key_]

2

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [33]:
%time bigram_phrases[sentences[78]]

TypeError: 'MySentences' object is not subscriptable

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [34]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [35]:
%time bigram_phraser[sentences[78]]

TypeError: 'MySentences' object is not subscriptable

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [36]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [37]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [ ]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [ ]:
print(corpus[:100])

## Entrainement d'un modèle Word2Vec sur ce corpus

In [ ]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [ ]:
outfile = f"../data/bulletins.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [19]:
model = Word2Vec.load("../data/bulletins.model")

### Imprimer le vecteur d'un terme

In [20]:
model.wv["bruxelles"]

array([-0.5748823 , -2.798214  , -1.1356295 ,  0.63985103,  1.6142458 ,
       -2.2673364 ,  0.7296653 , -0.3057057 ,  0.56751823,  2.2973344 ,
        0.38235128, -1.1501867 ,  2.0241442 ,  0.8259892 ,  0.67862934,
        1.1823063 , -2.1141653 , -1.2347897 , -2.520795  , -0.45474607,
        1.186323  , -3.679689  ,  0.40542   , -2.5815537 , -3.8172607 ,
       -1.5474522 , -0.82825714, -1.1680992 , -3.0549424 , -0.65157837,
       -2.5514934 , -0.10575458], dtype=float32)

### Calculer la similarité entre deux termes

In [21]:
model.wv.similarity("boucher", "boulanger")

0.65505946

### Chercher les mots les plus proches d'un terme donné

In [22]:
model.wv.most_similar("bruxelles", topn=10)

[('liege', 0.7310682535171509),
 ('nivelles', 0.7165259718894958),
 ('gand', 0.7111486196517944),
 ('hotel', 0.680022120475769),
 ('paris', 0.6632136702537537),
 ('saint_-_gilles', 0.6407042145729065),
 ('don', 0.6022297143936157),
 ('thotel', 0.5770599842071533),
 ('paix', 0.5765165090560913),
 ('londres', 0.5706126093864441)]

### Faire des recherches complexes à travers l'espace vectoriel

In [23]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('paris', 0.7126538157463074), ('nivelles', 0.7018144726753235), ('londres', 0.6549039483070374), ('liege', 0.650379478931427), ('gand', 0.6208880543708801), ('hotel', 0.58986496925354), ('prelude', 0.5890752673149109), ('faubourgs', 0.5886238813400269), ('mille_ames', 0.5848187804222107), ('vilvorde', 0.572725236415863)]
